<a href="https://colab.research.google.com/github/KrishnaVineel17/First_rep/blob/main/title_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import string
import numpy as np
import json

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

#load all the datasets 
from google.colab import files
uploaded = files.upload()
#
import io
#loading all jsons 
#load the datasets containing the category names
data1 = json.load(open('CA_category_id.json'))

uploaded = files.upload()
data2 = json.load(open('GB_category_id.json'))

uploaded = files.upload()
data3 = json.load(open('US_category_id.json'))

#loading all csv files
uploaded = files.upload()
df1 = pd.read_csv(io.BytesIO(uploaded['CAvideos.csv']))

uploaded = files.upload()
df2 = pd.read_csv(io.BytesIO(uploaded['GBvideos.csv']))

uploaded = files.upload()
df3 = pd.read_csv(io.BytesIO(uploaded['USvideos.csv']))


def category_extractor(data):
  i_d = [data['items'][i]['id'] for i in range(len(data['items']))]
  title = [data['items'][i]['snippet']["title"] for i in range(len(data['items']))]
  i_d = list(map(int, i_d))
  category = zip(i_d, title)
  category = dict(category)
  return category


#creating a new category column by mapping the category names to their ids
df1['category_title'] = df1['category_id'].map(category_extractor(data1))
df2['category_title'] = df2['category_id'].map(category_extractor(data2))
df3['category_title'] = df3['category_id'].map(category_extractor(data3))

#merge all df's
df = pd.concat([df1, df2, df3], ignore_index=True)

#drop rows based on duplicate videos
df = df.drop_duplicates('video_id')

#collect only titles of entertainment videos
#feel free to use any category of video that you want
entertainment = df[df['category_title'] == 'Entertainment']['title']
entertainment = entertainment.tolist()



#remove punctuations and convert text to lowercase
def clean_text(text):
    text = ''.join(e for e in text if e not in string.punctuation).lower()
    
    text = text.encode('utf8').decode('ascii', 'ignore')
    return text

corpus = [clean_text(e) for e in entertainment]

from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
  #get tokens
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index) + 1
 
  #convert to sequence of tokens
  input_sequences = []
  for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)
 
  return input_sequences, total_words
  
inp_sequences, total_words = get_sequence_of_tokens(corpus)


def generate_padded_sequences(input_sequences):
  max_sequence_len = max([len(x) for x in input_sequences])
  input_sequences = np.array(pad_sequences(input_sequences,  maxlen=max_sequence_len, padding='pre'))
  predictors, label = input_sequences[:,:-1], input_sequences[:, -1]
  label = ku.to_categorical(label, num_classes = total_words)
  return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

def create_model(max_sequence_len, total_words):
  input_len = max_sequence_len - 1
  model = Sequential()
 
  # Add Input Embedding Layer
  model.add(Embedding(total_words, 10, input_length=input_len))
 
  # Add Hidden Layer 1 — LSTM Layer
  model.add(LSTM(100))
  model.add(Dropout(0.1))
 
  # Add Output Layer
  model.add(Dense(total_words, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')
 
  return model
model = create_model(max_sequence_len, total_words)
model.fit(predictors, label, epochs=20, verbose=5)

def generate_text(seed_text, next_words, model, max_sequence_len):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1,  padding= 'pre')
    predicted = model.predict_classes(token_list, verbose=0)
 
  output_word = ""
  for word,index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += " "+output_word
  return seed_text.title()

print(generate_text("spiderman", 5, model, max_sequence_len))

Saving CA_category_id.json to CA_category_id.json


Saving GB_category_id.json to GB_category_id.json


Saving US_category_id.json to US_category_id.json


Saving CAvideos.csv to CAvideos.csv


Saving GBvideos.csv to GBvideos.csv


Saving USvideos.csv to USvideos.csv
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Spiderman Why


In [20]:
def category_extractor(data):
  i_d = [data['items'][i]['id'] for i in range(len(data['items']))]
  title = [data['items'][i]['snippet']["title"] for i in range(len(data['items']))]
  i_d = list(map(int, i_d))
  category = zip(i_d, title)
  category = dict(category)
  return category


#creating a new category column by mapping the category names to their ids
df1['category_title'] = df1['category_id'].map(category_extractor(data1))
df2['category_title'] = df2['category_id'].map(category_extractor(data2))
df3['category_title'] = df3['category_id'].map(category_extractor(data3))

#merge all df's
df = pd.concat([df1, df2, df3], ignore_index=True)

#drop rows based on duplicate videos
df = df.drop_duplicates('video_id')

#collect only titles of entertainment videos
#feel free to use any category of video that you want
entertainment = df[df['category_title'] == 'Entertainment']['title']
entertainment = entertainment.tolist()



#remove punctuations and convert text to lowercase
def clean_text(text):
    text = ''.join(e for e in text if e not in string.punctuation).lower()
    
    text = text.encode('utf8').decode('ascii', 'ignore')
    return text

corpus = [clean_text(e) for e in entertainment]

from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
  #get tokens
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index) + 1
 
  #convert to sequence of tokens
  input_sequences = []
  for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)
 
  return input_sequences, total_words
  
inp_sequences, total_words = get_sequence_of_tokens(corpus)


def generate_padded_sequences(input_sequences):
  max_sequence_len = max([len(x) for x in input_sequences])
  input_sequences = np.array(pad_sequences(input_sequences,  maxlen=max_sequence_len, padding='pre'))
  predictors, label = input_sequences[:,:-1], input_sequences[:, -1]
  label = ku.to_categorical(label, num_classes = total_words)
  return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

def create_model(max_sequence_len, total_words):
  input_len = max_sequence_len - 1
  model = Sequential()
 
  # Add Input Embedding Layer
  model.add(Embedding(total_words, 10, input_length=input_len))
 
  # Add Hidden Layer 1 — LSTM Layer
  model.add(LSTM(100))
  model.add(Dropout(0.1))
 
  # Add Output Layer
  model.add(Dense(total_words, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')
 
  return model
model = create_model(max_sequence_len, total_words)
model.fit(predictors, label, epochs=20, verbose=5)

def generate_text(seed_text, next_words, model, max_sequence_len):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1,  padding= 'pre')
    predicted = model.predict_classes(token_list, verbose=0)
 
  output_word = ""
  for word,index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += " "+output_word
  return seed_text.title()

print(generate_text("spiderman", 5, model, max_sequence_len))

NameError: ignored

NameError: ignored

NameError: ignored

NameError: ignored

NameError: ignored